In [1]:
# this notebook is to be executed on google colab
!pip install wget
import urllib 
from urllib import request
from PIL import Image
import io
from io import BytesIO
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt 
import wget
from google.colab import drive
drive.mount('/content/gdrive')

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=d47e153a546948f68b6d18f82c1baaccd72a91310c13fe008ac29c50da4d99f6
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
def load_getchu(dir):
    data = open(dir, 'r')
    data = data.read().split('\n')
    data = [x.split('\t') for x in data]
    return data

In [3]:
data = load_getchu('/content/getchu_firstrun_failed.txt')
data.reverse()
data = data[1:]
print(data)

[['http://www.getchu.com/soft.phtml?id=480925gc&gc=gc', '480925', '2008'], ['http://www.getchu.com/soft.phtml?id=563665gc&gc=gc', '563665', '2008'], ['http://www.getchu.com/soft.phtml?id=421925gc&gc=gc', '421925', '2007'], ['http://www.getchu.com/soft.phtml?id=725179gc&gc=gc', '725179', '2011'], ['http://www.getchu.com/soft.phtml?id=314125gc&gc=gc', '314125', '2007'], ['http://www.getchu.com/soft.phtml?id=457645gc&gc=gc', '457645', '2007'], ['http://www.getchu.com/soft.phtml?id=472985gc&gc=gc', '472985', '2007'], ['http://www.getchu.com/soft.phtml?id=457665gc&gc=gc', '457665', '2007'], ['http://www.getchu.com/soft.phtml?id=463805gc&gc=gc', '463805', '2007'], ['http://www.getchu.com/soft.phtml?id=461605gc&gc=gc', '461605', '2007'], ['http://www.getchu.com/soft.phtml?id=412525gc&gc=gc', '412525', '2007'], ['http://www.getchu.com/soft.phtml?id=625925gc&gc=gc', '625925', '2007'], ['http://www.getchu.com/soft.phtml?id=756987gc&gc=gc', '756987', '2009'], ['http://www.getchu.com/soft.phtml?id

Download images to ram or hdd.   
Images are named as 'gameID_chara_num'

In [4]:
def get_getchu_image_links(game_link):
    site = requests.get(game_link)
    soup = BeautifulSoup(site.content,"html.parser")
    links  = soup.findAll('img', style = 'border:none;')
    links = [ 'http://www.getchu.com/{}'.format(char.get('src')[1:]) for char in links]
    return links
def getchu_to_ram(data_part):
    game_link = data_part[0]
    links = get_getchu_image_links(game_link)
    opener = urllib.request.URLopener()
    opener.addheader('Referer', game_link)
    characters = [opener.open(link).read() for link in links]
    characters = [Image.open(BytesIO(data)) for data in characters]
    return characters
def getchu_to_hdd(data_part, dir):
    url = data_part[0]
    id = data_part[1]
    links = get_getchu_image_links(url)
    opener = urllib.request.URLopener()
    opener.addheader('Referer', url)
    index = 0
    for link in links:
      opener.retrieve(link, dir + '/{}char{}.jpg'.format(id,index))
      index += 1
    return 

In [ ]:
get_getchu_image_links(data[0][0])

['http://www.getchu.com//brandnew/224325/c224325chara1.jpg',
 'http://www.getchu.com//brandnew/224325/c224325chara2.jpg',
 'http://www.getchu.com//brandnew/224325/c224325chara3.jpg',
 'http://www.getchu.com//brandnew/224325/c224325chara4.jpg',
 'http://www.getchu.com//brandnew/224325/c224325chara5.jpg',
 'http://www.getchu.com//brandnew/224325/c224325chara6.jpg',
 'http://www.getchu.com//brandnew/224325/c224325chara7.jpg',
 'http://www.getchu.com//brandnew/224325/c224325chara8.jpg',
 'http://www.getchu.com//brandnew/224325/c224325chara9.jpg',
 'http://www.getchu.com//brandnew/224325/c224325chara10.jpg',
 'http://www.getchu.com//brandnew/224325/c224325chara11.jpg',
 'http://www.getchu.com//brandnew/224325/c224325chara12.jpg',
 'http://www.getchu.com//brandnew/224325/c224325chara13.jpg',
 'http://www.getchu.com//brandnew/224325/c224325chara14.jpg',
 'http://www.getchu.com//brandnew/224325/c224325chara15.jpg']

In [5]:
!cd content
!mkdir trying

/bin/bash: line 0: cd: content: No such file or directory


In [10]:
import threading, os , time
failed_links = []
def download_images(games, dir):
  for game in games:
    try:
     getchu_to_hdd(game, dir)
    except: 
      failed_links.append(game)
      print('link : {} has been failed'.format(game[0]))
  return
threads = []
num_threads = 14
img_per_thread = int(len(data)/ num_threads)
starting = time.time()
for i in range(num_threads):
  thread = threading.Thread( target = download_images, args = (data[i*img_per_thread:(i+1)*img_per_thread], '/content/trying') )
  thread.start()
  threads.append(thread)

while True: 
  time.sleep(10)
  sequence_finished = True
  for thread in threads: 
    sequence_finished &=  not thread.is_alive()
  if(sequence_finished):
    break
  else: 
    print('not yet')

finishing = time.time()
print('images are downloaded in {} seconds'.format(finishing - starting))
    


KeyboardInterrupt: ignored

In [8]:
len(failed_links)

0

In [ ]:
fails = open('/content/failed.txt','w')
s = ''
for f in failed_links:
  s += '{}\t{}\t{}\n'.format(f[0],f[1],f[2])
fails.write(s)

1764

In [ ]:
failed_links

[['http://www.getchu.com/soft.phtml?id=835407gc&gc=gc', '835407', '2014'],
 ['http://www.getchu.com/soft.phtml?id=635405gc&gc=gc', '635405', '2009'],
 ['http://www.getchu.com/soft.phtml?id=768962gc&gc=gc', '768962', '2013'],
 ['http://www.getchu.com/soft.phtml?id=1022990gc&gc=gc', '1022990', '2018'],
 ['http://www.getchu.com/soft.phtml?id=245665gc&gc=gc', '245665', '2006'],
 ['http://www.getchu.com/soft.phtml?id=683529gc&gc=gc', '683529', '2010'],
 ['http://www.getchu.com/soft.phtml?id=46001gc&gc=gc', '46001', '2004'],
 ['http://www.getchu.com/soft.phtml?id=770976gc&gc=gc', '770976', '2013'],
 ['http://www.getchu.com/soft.phtml?id=837360gc&gc=gc', '837360', '2014'],
 ['http://www.getchu.com/soft.phtml?id=679901gc&gc=gc', '679901', '2010'],
 ['http://www.getchu.com/soft.phtml?id=888260gc&gc=gc', '888260', '2016'],
 ['http://www.getchu.com/soft.phtml?id=711944gc&gc=gc', '711944', '2009'],
 ['http://www.getchu.com/soft.phtml?id=675581gc&gc=gc', '675581', '2010'],
 ['http://www.getchu.com/

In [ ]:
!zip -r /content/getchu_raw_pictures.zip /content/trying

Streaming output truncated to the last 5000 lines.
  adding: content/trying/585245char8.jpg (deflated 1%)
  adding: content/trying/86001char1.jpg (deflated 2%)
  adding: content/trying/591865char4.jpg (deflated 29%)
  adding: content/trying/1039861char4.jpg (deflated 2%)
  adding: content/trying/927420char4.jpg (deflated 1%)
  adding: content/trying/16183char4.jpg (deflated 4%)
  adding: content/trying/155121char1.jpg (deflated 3%)
  adding: content/trying/5780char1.jpg (deflated 5%)
  adding: content/trying/970303char1.jpg (deflated 0%)
  adding: content/trying/485585char4.jpg (deflated 29%)
  adding: content/trying/697744char1.jpg (deflated 2%)
  adding: content/trying/595945char8.jpg (deflated 13%)
  adding: content/trying/8080char4.jpg (deflated 4%)
  adding: content/trying/227665char0.jpg (deflated 27%)
  adding: content/trying/701782char2.jpg (deflated 3%)
  adding: content/trying/710906char9.jpg (deflated 3%)
  adding: content/trying/749995char22.jpg (deflated 2%)
  adding: cont